In [54]:
import pandas as pd
import numpy as np
# Retrieve path name matching patter
import glob

import json

import joblib

from tqdm import tqdm

In [55]:
root_path = 'data/'
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path)
meta_df.head(1)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998.0,unk,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125340?pdf=re...


In [56]:
pdf_json = glob.glob(f'{root_path}/**/pdf_json/*.json', recursive=True)
len(pdf_json)

45941

In [57]:
pmc_json = glob.glob(f'{root_path}/**/pmc_json/*.json', recursive=True)
len(pmc_json)

22263

In [58]:
import glob
all_json = glob.glob(f"{root_path}**/*.json",recursive=True)
len(all_json)

68204

In [59]:
class FileReader:
    def __init__(self,file_path):
        with open(file_path) as f:
            content = json.load(f)
            self.paper_id = content['paper_id']
            self.abstract =[]
            self.body_text =[]
            if "abstract" in content:
                for item  in  content['abstract']:
                    self.abstract.append(item['text']);
            for item in content['body_text']:
                self.body_text.append(item['text']);
            self.abstract =''.join(self.abstract)
            self.body_text =''.join(self.body_text)
    
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[0:200]} {self.body_text[0:200]}'
    
    def to_dict(self):
        return {'sha':self.paper_id, 'abstract':self.abstract,'body_text': self.body_text}

In [60]:
def readJson(json_files):
    data=[]
    for idx, file_path in enumerate(json_files):
        #if idx % (len(json_files) // 10) == 0:
            #print(f'Processing index: {idx} of {len(json_files)}')
        dict_data = FileReader(file_path).to_dict()
        data.append(dict_data)
    json_df = pd.DataFrame(data)
    return json_df

In [61]:
pmc_df = readJson(tqdm(pmc_json))

100%|██████████| 22263/22263 [00:32<00:00, 685.90it/s]


In [18]:
pdf_df = readJson(tqdm(pdf_json))

100%|██████████| 45941/45941 [01:07<00:00, 678.47it/s]


In [62]:
pdf_df.head()

,abstract,body_text,sha
0,The endoplasmic reticulum (ER) is the biggest ...,The endoplasmic reticulum (ER) is a multi-func...,86d4262de73cf81b5ea6aafb91630853248bff5f
1,,"Sir, Testing for respiratory virus infections ...",9ec0b1175992879d5b8d3351ef40a28bb48f18ce
2,The recent emergence of the Middle East respir...,While most CoVs cause the common cold in human...,86a998617c077f4fe2ab26214995a3548fbc0fc5
3,Graphical Abstract Highlights d MVS is a commo...,Clinically relevant respiratory viral signatur...,948aaeb2e0be11ad90562bf10d462531a1f00eac
4,,The Middle East respiratory syndrome coronavir...,4ed70c27f14b7f9e6219fe605eae2b21a229f23c


In [63]:
pmc_df.tail(10)

,abstract,body_text,sha
22253,,Upon the recognition of viral pathogen-associa...,PMC6706793
22254,,FHV-1 is a member of the Varicellovirus genus ...,PMC7020096
22255,,\nContact investigation (contact tracing) is t...,PMC4052784
22256,,"More than 200 different types of viruses, such...",PMC4033317
22257,,The pattern of recurrence of pandemics since t...,PMC3447299
22258,,Multivesicular bodies (MVBs) or late endosomes...,PMC4584306
22259,,"In recent years, emerging and re-emerging vira...",PMC6752855
22260,,Effective risk communication is an essential e...,PMC5053133
22261,,,PMC2276106
22262,,The worldwide epidemic of SARS (Severe Acute R...,PMC3509683


In [64]:
meta_df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998.0,unk,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125340?pdf=re...
1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,PMC,"Crystal structure of murine sCEACAM1a[1,4]: a ...",10.1093/emboj/21.9.2076,PMC125375,11980704.0,unk,CEACAM1 is a member of the carcinoembryonic an...,2002-05-01,"Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu...",The EMBO Journal,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125375?pdf=re...
2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,PMC,Synthesis of a novel hepatitis C virus protein...,10.1093/emboj/20.14.3840,PMC125543,11447125.0,no-cc,Hepatitis C virus (HCV) is an important human ...,2001-07-16,"Xu, Zhenming; Choi, Jinah; Yen, T.S.Benedict; ...",EMBO J,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,PMC,Structure of coronavirus main proteinase revea...,10.1093/emboj/cdf327,PMC126080,12093723.0,unk,The key enzyme in coronavirus polyprotein proc...,2002-07-01,"Anand, Kanchan; Palm, Gottfried J.; Mesters, J...",The EMBO Journal,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc126080?pdf=re...
4,i0zym7iq,dde02f11923815e6a16a31dd6298c46b109c5dfa,PMC,Discontinuous and non-discontinuous subgenomic...,10.1093/emboj/cdf635,PMC136939,12456663.0,unk,"Arteri-, corona-, toro- and roniviruses are ev...",2002-12-01,"van Vliet, A.L.W.; Smits, S.L.; Rottier, P.J.M...",The EMBO Journal,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc136939?pdf=re...


In [65]:
len(meta_df)

57366

In [66]:
# Drop blank text
pmc_df = pmc_df[pmc_df['body_text']!='']
pmc_df.rename(columns={"sha": "pmcid"}, inplace=True)
pmc_df.drop(columns =['abstract'], inplace=True)
pmc_df = pmc_df.merge(meta_df[['sha','pmcid']], how ='left', on = "pmcid")

In [67]:
df = pdf_df.merge(pmc_df, how="outer", on="sha")

In [68]:
df.describe()

,abstract,body_text_x,sha,body_text_y,pmcid
count,45941,45941,47019,19890,19890
unique,32283,45382,47019,19858,19890
top,,"In previous reports, workers have characterize...",9c33102dc55e225f5836f9cf9c9d5f5b918820d8,"The full text for this article, hosted at http...",PMC6262443
freq,12949,4,1,6,1


In [69]:
len_x =df['body_text_x'].fillna('').apply(len)
len_y = df['body_text_y'].fillna('').apply(len)
print(len(df[len_x>len_y]))
print(len(df[len_y>len_x]))

40412
7540


In [70]:
df[df.body_text_y.notnull() & df.body_text_x.isnull()]

,abstract,body_text_x,sha,body_text_y,pmcid
45941,NaN,NaN,NaN,Health authorities in Hong Kong are coming und...,PMC7094896
45942,NaN,NaN,NaN,The mRNA cap is a critical structure in gene e...,PMC7169794
45943,NaN,NaN,NaN,\nPlease visit methagora to view and post comm...,PMC7097306
45944,NaN,NaN,NaN,Dieser Beschluss soll natürlich die Möglichkei...,PMC7127843
45945,NaN,NaN,NaN,"Bemerkenswert ist zunächst, dass die KBV trotz...",PMC7127842
45946,NaN,NaN,NaN,"The third edition of this book, which was firs...",PMC7086707
45947,NaN,NaN,NaN,"In 2003, there were totally 8422 people worldw...",PMC7088578
45948,NaN,NaN,NaN,Malik Peiris really needs a holiday. The Hong ...,PMC7095930
45949,NaN,NaN,NaN,The linkages between facets of globalization a...,PMC2609866
45950,NaN,NaN,NaN,Pyrimidines and their annulated derivatives ex...,PMC7088342


In [71]:
# Both values not null and x string is bigger
# 12k entries
df[(len_x>len_y) & df.body_text_y.notnull() & df.body_text_x.notnull()]

,abstract,body_text_x,sha,body_text_y,pmcid
9,EV71) causes serious diseases in humans. The a...,Enteroviruses are members of the family Picorn...,44290ff75bad8ffaf5d3028420739ce7b08dc2a9,Enteroviruses are members of the family Picorn...,PMC7110181
11,"For analyzing the changes in immunoglobulins, ...",A World Health Organization (WHO) [1] alert in...,3c395c76b3d97ab269aed5e6c02d0f0184df7f9f,A World Health Organization (WHO) [1] alert in...,PMC7090797
16,Vaccines against infectious bronchitis of chic...,Severe acute respiratory syndrome (SARS) emerg...,7f1d647d43af4ed96625ed113f8bf0ab737e2f0c,Severe acute respiratory syndrome (SARS) emerg...,PMC7154303
17,Nanoscale materials can improve the effi cacy ...,"On the nanoscale, the chemical compositions, p...",11ef1846bb816e2884da201195b43ff8f4520cbb,"On the nanoscale, the chemical compositions, p...",PMC7169483
20,Purpose of Review Precision medicine (PM) repr...,Precision medicine (PM) is a growing up new pa...,23421cb20aecb11a6875ecaa1b4a9bbf698d560b,Precision medicine (PM) is a growing up new pa...,PMC7099688
29,Human papillomaviruses (HPVs) are formally des...,"Until the late 1970s, papillomaviruses attract...",bd3945ec343cc4535650c9425aa143f71716de7c,"Until the late 1970s, papillomaviruses attract...",PMC7108213
35,Bordetella bronchiseptica isolate KM22 has bee...,20-kb insert library preparation protocol (htt...,8f8eb4f004c2002face0723f2f58cc411954d36e,The whole-genome sequence for Bordetella bronc...,PMC6979298
37,Stressful exercise results in temporary immune...,"Introduction ssRNA viruses are influenza, noro...",4004362149e51fc0039454f0b4f5344bf5cac850,The nucleic acid in an RNA virus is usually si...,PMC7088253
47,Glia exhibit differential susceptibility to CD...,"In the quiescent central nervous system (CNS),...",cb6bc03f06dead8e79159e79f46ae11cfe218aff,"In the quiescent central nervous system (CNS),...",PMC7165990
53,"The common cold is the most frequent, although...",The common cold is an acute viral infection of...,21dc38640323fcbcaf547f3d05540db62c936f9d,The common cold is an acute viral infection of...,PMC7088639


In [72]:
# Both values not null and y string is bigger
# 5k entries
df[(len_x>len_y) & df.body_text_y.notnull() & df.body_text_x.notnull()]
df[(len_y>len_x) & df.body_text_y.notnull() & df.body_text_x.notnull()]

,abstract,body_text_x,sha,body_text_y,pmcid
4,,The Middle East respiratory syndrome coronavir...,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,The Middle East respiratory syndrome coronavir...,PMC7103731
94,The public health environment (like the rest o...,adjusting to local geographic conditions or ev...,ea7e0ab4e0dd488f8343630cb399d5333c610eb4,The public health environment (like the rest o...,PMC7124043
100,RNA interference (RNAi) is an evolutionarily c...,"well-known viruses, like the H1N1/09 influenza...",69faaacdd695a0b0d366b8bb3b98a94ce3176085,"Despite the advances in medical sciences, the ...",PMC7120812
103,Selective detection of α4β1 integrin (VLA-4)-e...,Significant improvements have been achieved in...,b84f49ede4b78f68a9e3a578fe667c06089ca3b6,Significant improvements have been achieved in...,PMC7167733
107,The influenza A nucleoprotein (NP) is an attra...,The transmission of a pathogenic avian H5N1 vi...,4df45b8404d9de0b376a8ae3c282a517df36fe51,The transmission of a pathogenic avian H5N1 vi...,PMC2820229
119,,Respiratory infections in animal species are a...,68e351f6a8d6bc2b34edfc6ef03143f5ab42d6c2,Respiratory infections in animal species are a...,PMC7123793
149,Biotechnology uses living systems to develop p...,. Plant cell culture systems represent a poten...,e03bb3f2c96cd9bad3cd08e8b48fcf0d34f103a6,Molecular biotechnology is the use of laborato...,PMC7123938
150,Porcine-transmissible gastroenteritis virus (T...,Porcine-transmissible gastroenteritis virus (T...,84bc6f34aed1ffa36fd8ee698e37997e926d30f4,Porcine diarrhoea can lead to weight loss and ...,PMC7169721
156,,Improving the care of adult patients with comm...,6bc54eaa333c5ded1fe01b1aadbf79c0b6bbb4f2,Improving the care of adult patients with comm...,PMC7107997
168,Computed tomography (CT) imaging is useful in ...,Seizures were the most common signs in five of...,fa1bffd886207905da8201a97a3d532f01d3d3d6,Computed tomography (CT) imaging is useful in ...,PMC7159546


In [30]:
#df.loc[df.body_text_y.notnull(), 'body_text_x']  = df.loc[df.body_text_y.notnull(), 'body_text_y']

In [31]:
#text_formatted = [re.sub('[^a-zA-z0-9\s]','',word) for word in text.split(' ')]
# Abstract x, y seems same most of the cases but y seems to better fomrmatted, 
# So we take body text as y whenever possible

In [73]:
df['body_text'] = df['body_text_x']
df.loc[df.body_text_y.notnull(), 'body_text']  = df.loc[df.body_text_y.notnull(), 'body_text_y']

In [74]:
data_df = df.drop(['body_text_x','body_text_y','pmcid'], axis=1)

In [75]:
data_df.head()

,abstract,sha,body_text
0,The endoplasmic reticulum (ER) is the biggest ...,86d4262de73cf81b5ea6aafb91630853248bff5f,The endoplasmic reticulum (ER) is a multi-func...
1,,9ec0b1175992879d5b8d3351ef40a28bb48f18ce,"Sir, Testing for respiratory virus infections ..."
2,The recent emergence of the Middle East respir...,86a998617c077f4fe2ab26214995a3548fbc0fc5,While most CoVs cause the common cold in human...
3,Graphical Abstract Highlights d MVS is a commo...,948aaeb2e0be11ad90562bf10d462531a1f00eac,Clinically relevant respiratory viral signatur...
4,,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,The Middle East respiratory syndrome coronavir...


In [35]:
len(data_df)

47963

In [76]:
meta_df.dropna(subset=['sha','abstract'], inplace=True)

In [77]:
len(meta_df[meta_df.duplicated(['sha'],keep=False)])

14

In [78]:
meta_df[meta_df.duplicated(['sha'],keep=False)].sort_values(by="sha")

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
16418,mmls866r,151b80cfa218d8a252c1720d3e2f2e7b1effd96c,PMC,Clinical virology in real time,10.1016/s1386-6532(02)00197-x,PMC7128152,12126716.0,no-cc,The ability to detect nucleic acids has had an...,2002-12-03,"Niesters, Hubert G.M",J Clin Virol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
29346,mmls866r,151b80cfa218d8a252c1720d3e2f2e7b1effd96c,Elsevier,Clinical virology in real time,10.1016/s1386-6532(02)00197-x,NaN,12467772.0,els-covid,Abstract The ability to detect nucleic acids h...,2002-12-31,"Niesters, Hubert G.M",Journal of Clinical Virology,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/s1386-6532(02)00197-x
6987,7y8fd521,58b578c2c6619df320c9a0bcb4d434edc2300d00,PMC,Gene expression patterns in peripheral blood l...,10.1016/j.hal.2016.03.009,PMC5008692,27594814.0,no-cc,Ciguatera fish poisoning (ciguatera) is a comm...,2016-08-30,"Lopez, Maria-Cecilia; Ungaro, Ricardo F.; Bake...",Harmful Algae,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
29564,7y8fd521,58b578c2c6619df320c9a0bcb4d434edc2300d00,Elsevier,Gene expression patterns in peripheral blood l...,10.1016/j.hal.2016.03.009,NaN,28918889.0,els-covid,Abstract Ciguatera fish poisoning (ciguatera) ...,2016-07-31,"Lopez, Maria-Cecilia; Ungaro, Ricardo F.; Bake...",Harmful Algae,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.hal.2016.03.009
16398,e9pwguwm,70e2b820b3d2056eb90ac0e11ac230267fd0d7e8,PMC,Microbicides and the environmental control of ...,10.1016/j.jhin.2003.12.033,PMC7124243,14706273.0,no-cc,Viruses are important causes of acute and chro...,2004-04-02,"Sattar, S.A",J Hosp Infect,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
28959,e9pwguwm,70e2b820b3d2056eb90ac0e11ac230267fd0d7e8,Elsevier,Microbicides and the environmental control of ...,10.1016/j.jhin.2003.12.033,NaN,15110126.0,els-covid,Abstract Viruses are important causes of acute...,2004-04-30,"Sattar, S.A",Journal of Hospital Infection,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.jhin.2003.12.033
16430,8fwa2c24,77943f83d13697f86b9d1eb3cfa86581ed9965e6,PMC,From Hippocrates to HIPAA: Privacy and confide...,10.1016/j.annemergmed.2004.08.008,PMC7132445,15635311.0,no-cc,Respect for patient privacy and confidentialit...,2004-12-01,"Moskop, John C.; Marco, Catherine A.; Larkin, ...",Ann Emerg Med,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
28972,8fwa2c24,77943f83d13697f86b9d1eb3cfa86581ed9965e6,Elsevier,From Hippocrates to HIPAA: Privacy and confide...,10.1016/j.annemergmed.2004.08.008,NaN,15635312.0,els-covid,Respect for patient privacy and confidentialit...,2005-01-31,"Moskop, John C.; Marco, Catherine A.; Larkin, ...",Annals of Emergency Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.annemergmed.2004.08.008
16419,0z5wacxs,7e787fd2ae5b544add6281d3d40ad322de26aa17,PMC,Transportation capacity for patients with high...,10.1111/1469-0691.12290,PMC7128608,25636943.0,no-cc,Highly infectious diseases (HIDs) are defined ...,2015-06-22,"Schilling, S.; Maltezou, H.C.; Fusco, F.M.; De...",Clin Microbiol Infect,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
29293,0z5wacxs,7e787fd2ae5b544add6281d3d40ad322de26aa17,Elsevier,Transportation capacity for patients with high...,10.1111/1469-0691.12290,NaN,24750421.0,els-covid,Abstract Highly infectious diseases (HIDs) are...,2019-04-30,"Schilling, S.; Maltezou, H.C.; Fusco, F.M.; De...",Clinical Microbiology and Infection,NaN,NaN,True,False,custom_license,https://doi.org/10.1111/1469-0691.12290


In [79]:
# Drop duplicate entries in meta data
duplicates =meta_df[meta_df.duplicated(['sha'], keep=False)]
duplicates.index
meta_df.drop(duplicates.index, inplace=True)

In [80]:
data_df.describe()

,abstract,sha,body_text
count,45941,47019,47963
unique,32283,47019,47388
top,,9c33102dc55e225f5836f9cf9c9d5f5b918820d8,"The full text for this article, hosted at http..."
freq,12949,1,6


In [81]:
data_df

,abstract,sha,body_text
0,The endoplasmic reticulum (ER) is the biggest ...,86d4262de73cf81b5ea6aafb91630853248bff5f,The endoplasmic reticulum (ER) is a multi-func...
1,,9ec0b1175992879d5b8d3351ef40a28bb48f18ce,"Sir, Testing for respiratory virus infections ..."
2,The recent emergence of the Middle East respir...,86a998617c077f4fe2ab26214995a3548fbc0fc5,While most CoVs cause the common cold in human...
3,Graphical Abstract Highlights d MVS is a commo...,948aaeb2e0be11ad90562bf10d462531a1f00eac,Clinically relevant respiratory viral signatur...
4,,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,The Middle East respiratory syndrome coronavir...
5,Thousands of people in the United States have ...,306ef95a3a91e13a93bcc37fb2c509b67c0b5640,"The 2019 novel coronavirus (SARS-CoV-2), ident..."
6,,ab680d5dbc4f51252da3473109a7885dd6b5eb6f,The evolutionary history of humans is characte...
7,"International aviation is growing rapidly, res...",6599ebbef3d868afac9daa4f80fa075675cf03bc,"Sixty years ago, civil aviation was an infant ..."
8,,eb5c7f3ff921ad6469b79cc8a3c122648204ece4,Acute infections of the gastrointestinal tract...
9,EV71) causes serious diseases in humans. The a...,44290ff75bad8ffaf5d3028420739ce7b08dc2a9,Enteroviruses are members of the family Picorn...


In [83]:
data_df = data_df.merge(meta_df[['title','abstract','sha','url','doi','publish_time','authors','journal']], how="left", on="sha")
data_df.head()

,abstract_x,sha,body_text,title,abstract_y,url,doi,publish_time,authors,journal
0,The endoplasmic reticulum (ER) is the biggest ...,86d4262de73cf81b5ea6aafb91630853248bff5f,The endoplasmic reticulum (ER) is a multi-func...,Urban planning of the endoplasmic reticulum (E...,Abstract The endoplasmic reticulum (ER) is the...,https://doi.org/10.1016/j.bbamcr.2011.06.011,10.1016/j.bbamcr.2011.06.011,2011-10-31,"Lynes, Emily M.; Simmen, Thomas",Biochimica et Biophysica Acta (BBA) - Molecula...
1,,9ec0b1175992879d5b8d3351ef40a28bb48f18ce,"Sir, Testing for respiratory virus infections ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The recent emergence of the Middle East respir...,86a998617c077f4fe2ab26214995a3548fbc0fc5,While most CoVs cause the common cold in human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Graphical Abstract Highlights d MVS is a commo...,948aaeb2e0be11ad90562bf10d462531a1f00eac,Clinically relevant respiratory viral signatur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,The Middle East respiratory syndrome coronavir...,Update on therapeutic options for Middle East ...,Introduction: The Middle East Respiratory Synd...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,10.1080/14787210.2017.1271712,2016-12-24,"Al-Tawfiq, Jaffar A.; Memish, Ziad A.",Expert Rev Anti Infect Ther


In [84]:
#abstract_y has abstract key word in front
def formatAbstract(abstract):
    if(pd.notna(abstract) and abstract.split(' ')[0].lower() == "abstract"):
        return abstract[9:] 
    else:
        return abstract
data_df['abstract_y'] =data_df['abstract_y'].apply(formatAbstract)

In [85]:
data_df.head()

,abstract_x,sha,body_text,title,abstract_y,url,doi,publish_time,authors,journal
0,The endoplasmic reticulum (ER) is the biggest ...,86d4262de73cf81b5ea6aafb91630853248bff5f,The endoplasmic reticulum (ER) is a multi-func...,Urban planning of the endoplasmic reticulum (E...,The endoplasmic reticulum (ER) is the biggest ...,https://doi.org/10.1016/j.bbamcr.2011.06.011,10.1016/j.bbamcr.2011.06.011,2011-10-31,"Lynes, Emily M.; Simmen, Thomas",Biochimica et Biophysica Acta (BBA) - Molecula...
1,,9ec0b1175992879d5b8d3351ef40a28bb48f18ce,"Sir, Testing for respiratory virus infections ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The recent emergence of the Middle East respir...,86a998617c077f4fe2ab26214995a3548fbc0fc5,While most CoVs cause the common cold in human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Graphical Abstract Highlights d MVS is a commo...,948aaeb2e0be11ad90562bf10d462531a1f00eac,Clinically relevant respiratory viral signatur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,The Middle East respiratory syndrome coronavir...,Update on therapeutic options for Middle East ...,Introduction: The Middle East Respiratory Synd...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,10.1080/14787210.2017.1271712,2016-12-24,"Al-Tawfiq, Jaffar A.; Memish, Ziad A.",Expert Rev Anti Infect Ther


In [86]:
data_df['abstract'] = data_df['abstract_x']

In [87]:
data_df.loc[data_df.abstract_y.notnull(), 'abstract'] = data_df.loc[data_df.abstract_y.notnull(), 'abstract_y']

In [89]:
data_df.head()

,abstract_x,sha,body_text,title,abstract_y,url,doi,publish_time,authors,journal,abstract
0,The endoplasmic reticulum (ER) is the biggest ...,86d4262de73cf81b5ea6aafb91630853248bff5f,The endoplasmic reticulum (ER) is a multi-func...,Urban planning of the endoplasmic reticulum (E...,The endoplasmic reticulum (ER) is the biggest ...,https://doi.org/10.1016/j.bbamcr.2011.06.011,10.1016/j.bbamcr.2011.06.011,2011-10-31,"Lynes, Emily M.; Simmen, Thomas",Biochimica et Biophysica Acta (BBA) - Molecula...,The endoplasmic reticulum (ER) is the biggest ...
1,,9ec0b1175992879d5b8d3351ef40a28bb48f18ce,"Sir, Testing for respiratory virus infections ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,The recent emergence of the Middle East respir...,86a998617c077f4fe2ab26214995a3548fbc0fc5,While most CoVs cause the common cold in human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The recent emergence of the Middle East respir...
3,Graphical Abstract Highlights d MVS is a commo...,948aaeb2e0be11ad90562bf10d462531a1f00eac,Clinically relevant respiratory viral signatur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Graphical Abstract Highlights d MVS is a commo...
4,,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,The Middle East respiratory syndrome coronavir...,Update on therapeutic options for Middle East ...,Introduction: The Middle East Respiratory Synd...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,10.1080/14787210.2017.1271712,2016-12-24,"Al-Tawfiq, Jaffar A.; Memish, Ziad A.",Expert Rev Anti Infect Ther,Introduction: The Middle East Respiratory Synd...


In [91]:
data_df.drop(columns=['abstract_x','abstract_y'], inplace=True)

In [94]:
data_df[data_df.duplicated(subset=['body_text'], keep=False)]

,sha,body_text,title,url,doi,publish_time,authors,journal,abstract
3,948aaeb2e0be11ad90562bf10d462531a1f00eac,Clinically relevant respiratory viral signatur...,NaN,NaN,NaN,NaN,NaN,NaN,Graphical Abstract Highlights d MVS is a commo...
73,d1a0cb9ae34183dbb37a0d87d3c4c690a0341696,Severe acute respiratory syndrome (SARS) is a ...,NaN,NaN,NaN,NaN,NaN,NaN,A combinatorial human immunoglobulin gene libr...
88,12f1cc78e99a0b82f3225d10abe9a5649f1b33fc,Coronaviruses comprise a genus of the family C...,NaN,NaN,NaN,NaN,NaN,NaN,Little is known about the prevalence of mucosa...
98,4f0d3c7d8a703aeef06f5cbccfa1244a768f7aed,Recent publications have brought attention to ...,NaN,NaN,NaN,NaN,NaN,NaN,
112,0d9c07aab17cf3386f137763c18387483fe36227,One major distinction between HIV-1 and the le...,NaN,NaN,NaN,NaN,NaN,NaN,"Vpu, a component unique to HIV-1, greatly enha..."
170,162e2d7f1b49b31893a155877f8699fb8a87082e,The causative agent of the severe acute respir...,NaN,NaN,NaN,NaN,NaN,NaN,The coronavirus responsible for the severe acu...
240,a51943a6b6527dd8f62bc27c061a6371fd045dce,The SARS-CoV nucleocapsid protein (NP) is a ma...,NaN,NaN,NaN,NaN,NaN,NaN,"Differential scanning calorimetry, circular di..."
310,34ba72963a44bbe5ce71b7ec96c6f71dd1069c2e,p 5 0.005) prevalence in children who needed t...,NaN,NaN,NaN,NaN,NaN,NaN,"RATIONALE: In infancy, rhinovirus illnesses an..."
311,e6e0128e5f23630f1be605cb1a7fb4cc8261f323,The unifying theme of my research centers on t...,NaN,NaN,NaN,NaN,NaN,NaN,The review centers on the human gastrointestin...
315,c9db6fb06b3207d4f1d753e38c34941b1a6fdcc7,mens. Sputum was diluted with an equal volume ...,NaN,NaN,NaN,NaN,NaN,NaN,S ystemic glucocorticosteroids (steroids) are ...


In [95]:
data_df.drop_duplicates(subset=['body_text'], keep='first', inplace=True)

In [96]:
data_df[data_df.duplicated(subset=['body_text'], keep=False)]

,sha,body_text,title,url,doi,publish_time,authors,journal,abstract


In [97]:
data_df.describe()

,sha,body_text,title,url,doi,publish_time,authors,journal,abstract
count,46451,47388,36622,36657,36363,36657,36444,34172,46418
unique,46451,47388,36390,36657,36363,5844,35549,5401,39985
top,9c33102dc55e225f5836f9cf9c9d5f5b918820d8,"sibilantes y trabajo respiratorio, mayor durac...",Infectious Diseases,https://doi.org/10.1016/j.tibtech.2007.10.001,10.1016/j.iac.2015.12.001,2016-12-31,"Domingo, Esteban",PLoS One,
freq,1,1,7,1,1,205,14,1569,5873


In [98]:
# Only repeated title is NaN
data_df[data_df.duplicated(subset=['abstract'], keep=False)].sort_values('abstract')

,sha,body_text,title,url,doi,publish_time,authors,journal,abstract
1,9ec0b1175992879d5b8d3351ef40a28bb48f18ce,"Sir, Testing for respiratory virus infections ...",NaN,NaN,NaN,NaN,NaN,NaN,
24084,4a9d97e763598cc4a6d0a99013b4493e42db321d,Editorial 50th Anniversary Editorial: Building...,NaN,NaN,NaN,NaN,NaN,NaN,
24082,3c928eef1ab9e10c33a73c0dec240b088b82c06f,\nIntroduction: The provision of an effective ...,NaN,NaN,NaN,NaN,NaN,NaN,
24080,f9164b23f6353a2e3c4c7380280ab9bdc20d2875,CoronaVirus Disease-19 (COVID-19) poses a thre...,NaN,NaN,NaN,NaN,NaN,NaN,
24079,273793fabfa7023132df275fca4fdc13fade3684,EMERGING TECHNOLOGY: IONIC LIQUIDS N ame an ac...,NaN,NaN,NaN,NaN,NaN,NaN,
24076,70fb71351caca2ce4100b806ac9d99fa094f8453,"Family Foundation, Health Coverage of Immirgan...",NaN,NaN,NaN,NaN,NaN,NaN,
24072,e4514865ab32606bac0018ad579fe143067874bd,Following mass population displacements in Sou...,NaN,NaN,NaN,NaN,NaN,NaN,
24069,c6a131a5f2adb1cff23f1cc7325354f6d3b6c139,Canada's celebration in 2017 of 150 years as a...,NaN,NaN,NaN,NaN,NaN,NaN,
24066,9c23fe1540707c4a223cdd8d2acaf8cbb175df6a,"""If you know the enemy and know yourself, you ...",NaN,NaN,NaN,NaN,NaN,NaN,
24057,f2331a5c6268ec47649983001535019aff087c35,The host evokes innate immune responses to eli...,NaN,NaN,NaN,NaN,NaN,NaN,


In [99]:
data_df['body_len'] = data_df['body_text'].apply(len)

In [100]:
data_df.to_csv('covid_df.csv', index=False)

In [ ]:
data_df[data_df['body_len'] ==data_df['body_len'].max()]

In [ ]:
data_df.loc[46180]['body_text']